# Rubin LSST DESC DC2: Accessing Truth-match Table with GCRCatalogs

**Authors**: Yao-Yuan Mao (@yymao), Javier Sanchez (@fjaviersanchez), Joanne Bogart (@JoanneBogart)

This notebook will illustrate the basics of accessing the Truth-match Table, which contains the basic properties of truth objects (i.e., inputs to the image simulations) and also matching information with respect to the Object Table.

**Prerequisite**: Please go over the Object Table tutorial first!  

**Learning objectives**: After going through this notebook, you should be able to:
  1. Load and efficiently access a DC2 Truth-match Table with the GCRCatalogs
  2. Understand and have references for the Truth-match Table schema and data model
  3. Make some validation plots using the Truth-match Table

## Import necessary packages

In [ ]:
import sys
sys.path.insert(0, "/global/homes/y/yymao/desc/generic-catalog-reader/")
sys.path.insert(0, "/global/homes/y/yymao/desc/gcr-catalogs/")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import GCRCatalogs
from GCRCatalogs.helpers.tract_catalogs import tract_filter, sample_filter
from GCRCatalogs import GCRQuery

## Access Truth-match Table with GCRCatalogs

Just like the Object Table, the Truth-match Table is available as parquet files, 
and `GCRCatalogs` provides a high-level user interface for accessing it. 

However, while the Truth-match Table is a single set of parquet files, it appears as two entries in `GCRCatalogs` as we will see below. 

- The catalog entry `desc_dc2_run2.2i_dr6_truth` contains all truth information, including true sources that are not detected in the DR6 Object Table. 
- The catalog entry `desc_dc2_run2.2i_dr6_object_with_truth_match` contains all the identical information of the Object Table (`desc_dc2_run2.2i_dr6_object`), plus additional columns that contain information of best-match truth objects. In other words, `desc_dc2_run2.2i_dr6_object_with_truth_match` is like a "left join" of `desc_dc2_run2.2i_dr6_object` and `desc_dc2_run2.2i_dr6_truth`, but with spatial separation and magnitude difference as join keys. See the release note for further details. 

### Which catalog entry should I use?

- If you don't need any truth information, just use `desc_dc2_run2.2i_dr6_object`. 
- If you only need the best-match truth source information for each object (i.e., each row) in the Object Table, use `desc_dc2_run2.2i_dr6_object_with_truth_match`
- If you need more truth infomation beyond the best matches (i.e., undetected sources, blended sources), you need to use `desc_dc2_run2.2i_dr6_truth`

In [ ]:
GCRCatalogs.get_public_catalog_names()

In [ ]:
GCRCatalogs.set_root_dir("/global/cfs/cdirs/lsst/gsharing/dc2_public/lsstdesc-dc2-wfd-dr6-v1")

In [ ]:
object_cat = GCRCatalogs.load_catalog('desc_dc2_run2.2i_dr6_object')

In [ ]:
match_cat = GCRCatalogs.load_catalog('desc_dc2_run2.2i_dr6_object_with_truth_match')

In [ ]:
truth_cat = GCRCatalogs.load_catalog('desc_dc2_run2.2i_dr6_truth')

Let's first take a look at `match_cat` (which loads `desc_dc2_run2.2i_dr6_object_with_truth_match`). 
As discussed earlier, this catalog is just Object Table plus best match truth information. 
We can check if all the columns in the Object Table are present in this catalog too.

In [ ]:
set(object_cat.list_all_quantities()).issubset(match_cat.list_all_quantities())

Indeed `match_cat` contains all columns in the Object Table. So what are the additional columns? 

In [ ]:
set(match_cat.list_all_quantities()).difference(object_cat.list_all_quantities())

These columns describe the infomation of the matched truth source for each object in the Object Table. 
Note that most of these columns have a `_truth` postfix (i.e., suffix) to avoid potential confusion with columns in the Object Table.
In fact, these columns are the same set of columns in `truth_cat` (but there's no `_truth` postfix in `truth_cat`):

In [ ]:
sorted(truth_cat.list_all_quantities())

In [ ]:
# We can verify what we just said is indeed true:

additional_cols_in_match = set(match_cat.list_all_quantities()).difference(object_cat.list_all_quantities())
set(col[:-6] if col.endswith("_truth") else col for col in additional_cols_in_match) == set(truth_cat.list_all_quantities())

### What are "best matches"?

Recall that `match_cat` is a "left join" of Object Table and Truth Table, so each object is assigned a "best match" truth source.
A "best match" truth source is the truth source that is within 1 arcsec of the object in consideration, and has the smallest magnitude difference.
If there's no truth sources that are within 1 arcsec of the object in consideration, or if the smallest magnitude difference is larger than 1 mag, 
then the "best match" would be just the nearest neighbor. 

If a best match satisfies the two criteria (separation < 1 arcsec and magnitude difference <  1 mag), we call it a "good match". Otherwise, it would just be the nearest neighbor. Note that a "good match" could still (in fact, is likely to) be the nearest neighbor. 

Let's take a closer look. We can use the `is_good_match` and `is_nearest_neighbor` to tell whether the best match is a good match and/or the nearest neighbor.

In [ ]:
data = match_cat.get_quantities(["mag_r_cModel", "mag_r_truth", "match_sep", "is_good_match", "is_nearest_neighbor"], native_filters=[tract_filter(3830)])

In [ ]:
# Note that if the best match is not a good match, then it must be the nearest neighbor.

(data["is_good_match"] | data["is_nearest_neighbor"]).all()

In [ ]:
# Most "good matches" (sep. < 1 arcsec, dmag < 1 mag) would also happen to be nearest neighbors (NN). 
# "Not good" matches can be due to either large separation or large magnitude difference. 

# Let's define a few masks to distinguish these cases:

dmag = np.abs(data["mag_r_truth"] - data["mag_r_cModel"])

good_and_nn = data["is_good_match"] & data["is_nearest_neighbor"]     # Good matches that happen to be the nearest neighbor (NN) too
good_not_nn = data["is_good_match"] & (~data["is_nearest_neighbor"])  # Good matches that are not the nearest neighbor (NN)
not_good_small_dmag = (~data["is_good_match"]) & (dmag < 1)           # Not good matches that have dmag < 1 mag (so they must have sep > 1 arcsec)
not_good_large_dmag = (~data["is_good_match"]) & ((dmag >= 1) | np.isnan(dmag))  # Not good matches that have dmag > 1 mag or undefined mag

In [ ]:
# These four masks are mutually exclusive:

(np.vstack([good_and_nn, good_not_nn, not_good_small_dmag, not_good_large_dmag]).astype(np.int).sum(axis=0) == 1).all()

In [ ]:
sep_bins = np.linspace(0, 5, 51)
fig, ax = plt.subplots(ncols=2, figsize=(10, 4), dpi=100)

hist = ax[0].hist(data["match_sep"], bins=sep_bins)[0]
ax[0].set_yscale("log");
ax[0].set_xlabel("separation [arcsec]");
ax[0].set_ylabel("number (per bin)");


bottom = None
for mask, color in zip(
    [good_and_nn, good_not_nn, not_good_small_dmag, not_good_large_dmag],
    [plt.cm.Greens(0.8), plt.cm.Greens(0.4), plt.cm.Oranges(0.4), plt.cm.Oranges(0.8)]
):
    frac = np.histogram(data["match_sep"][mask], bins=sep_bins)[0] / hist
    ax[1].bar(sep_bins[:-1], frac, width=np.ediff1d(sep_bins), bottom=bottom, align="edge", color=color)
    if bottom is None:
        bottom = frac
    else:
        bottom += frac
        
ax[1].set_xlabel("separation [arcsec]");
ax[1].set_ylabel("fraction (per bin)");

In the figure above, the left panel shows the distribution of spatial separation for all best matches (note that y-axis is in log scale). 
The vast majority of objects have matches within 1 arcsec, with a small tail goes to up to 5 arcsec. 

The right panel shows that, in each bin of spatial separation, the fraction of the four situations that we defined early. 
The greens bars denote "good matches" (sep. < 1 arcsec and dmag < 1 mag), and we can see that the majority of "good matches" are also 
nearest neighbor (darker green), especially if the separation is small. 
The oranges bars denote "not good matches" (so that they are all nearest neighbors by definition).
About only 20% of these nearest neighbors but not good matches have close magnitude (dmag < 1 mag). 

Let's also look at the measured vs. true magnitudes for these different matches. 

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(8, 8), dpi=100)

for mask, color, label, ax_this in zip(
    [good_and_nn, good_not_nn, not_good_small_dmag, not_good_large_dmag],
    [plt.cm.Greens(0.8), plt.cm.Greens(0.4), plt.cm.Oranges(0.4), plt.cm.Oranges(0.8)],
    ["good match\nnearest neighbor", "good match\nnot nearest neighbor", "not good match\nnearest neighbor\ndmag < 1", "not good match\nnearest neighbor\ndmag > 1"],
    ax.flat
):
    ax_this.scatter(data["mag_r_truth"][mask], data["mag_r_cModel"][mask], color=color, s=0.005, rasterized=True)
    ax_this.set_xlim(15, 35)
    ax_this.set_ylim(15, 35)
    ax_this.plot([15, 35], [15, 35], color='grey', lw=0.5)
    ax_this.text(16, 34, label, va="top")
    
fig.text(0.5, 0.02, "true r-band mag", ha="center");
fig.text(0.02, 0.5, "measured r-band mag", va="center", rotation=90);


Note that because `match_cat` uses the Object Table as the reference catalog for the match, a unique truth source may appear twice in the catalog.

In [ ]:
truth_id = match_cat.get_quantities(["id_truth"], native_filters=tract_filter(3830))["id_truth"]

number_of_objects = len(truth_id)
number_of_unique_truth = len(np.unique(truth_id))

print("Number of objects", number_of_objects)
print("Number of unique truth sources", number_of_unique_truth)

### Truth Table

Now we switch to look at the full truth table `truth_cat`. When using `truth_cat`, you will obtain all truth sources, including those below the detection limit or blended. You will also obtain only *unique* truth sources (unlike the case of `match_cat` as we shown above). 

The Truth Table allows us to inspect, for example, undetected sources. 
Here, let's look at how many truth galaxies are detected (and not blended) as a function of the galaxy magnitude. 

To select only galaxies, we can filter on `truth_type == 1`. Here the truth type is 1 for galaxies, 2 for stars, and 3 for SNe. 

In [ ]:
truth_galaxies = truth_cat.get_quantities(
    quantities=["match_objectId", "is_good_match", "mag_r"], 
    filters=["truth_type == 1"], 
    native_filters=[tract_filter(3830)],
)

In [ ]:
mag_bins = np.linspace(14.5, 29.5, 46)

fig, ax = plt.subplots(figsize=(6,4), dpi=100)

ax.hist(truth_galaxies["mag_r"][truth_galaxies["is_good_match"]], mag_bins, histtype="step", label="has good match", lw=1.5, color="C2");
ax.hist(truth_galaxies["mag_r"][(~truth_galaxies["is_good_match"]) & (truth_galaxies["match_objectId"] > -1)], mag_bins, histtype="step", label="has NN (but not good) match", lw=1.5, color="C1");
ax.hist(truth_galaxies["mag_r"][truth_galaxies["match_objectId"] == -1], mag_bins, histtype="step", label="has no matches at all", lw=1.5, color="C3");

ax.set_yscale("log");
ax.legend(loc="upper left");
ax.set_xlabel("r-band mag");
ax.set_ylabel("number per bin");
ax.set_xticks(np.arange(14, 31));

From the plot above, we see that at around $r = 27$, about half of the sources are "detected", in the sense that a good match in the Object Table is found. Much more sources fainter than $r = 27$ are not detected (or a match is not found). 

On the bright end, the vast majority have good matches. A handful cases where there is no good matches or no matches are interesting blending study cases.